# Texture synthesis and artistic style transfer

In this homework you are to imlement [A Neural algorithm of artistic style](http://arxiv.org/pdf/1508.06576v2.pdf). This is an extension of [Texture Synthesis Using Convolutional Neural Networks](http://arxiv.org/pdf/1505.07376v3.pdf) method.

The core of the method -- VGG and constrained optimization. The constrains are of two types: *content* and *style*. Given a content image **C** and style image **S** we want to generate an image **X** with content from **C** and style (whatever it really means) from **S**. 

We want to design a loss function for the optimization process. Considering \[1\], \[2\], an input image is easily invertable from the outputs at intermediate layers. This explains the idea of making an intermediate representation $F_X$ of **X** close to **C** representation $F_C$. 

$$
   L_{content} = || F_X - F_C || \rightarrow \min_X
$$

Note, that representation $F$ preserve spatial information. Idea: let us dismiss it, so we will know what objects are there on the picture, but will not be able to reestablish their localtion. The style can be thought as something independent of content, something we are left with if we let the content off. L. Gatys suggests to dismiss spatial information by computing correlations between the feature maps $F$. If $F$ has dimensions `CxWxH`, then correlation matrix will be `CxC`, and look there's no spatial dimentions. So the style term will be responsible for mathing these correlation (Gram) matrices. 

$$
   L_{style} = || Gram(F_X) - Gram(F_C) || \rightarrow \min_X
$$

And finaly we combine the two.

$$
   L = \alpha L_{content} + \beta L_{style} \min_X
$$

Read the paper and the code for the details on layers, features $F$ are got from.

#### A little bit of history behind this texture generation method

Actually the idea comes from 90th, when mathematical models of texures were developed \[3\]. They defined a probabolistic model for texture generation. They used an idea, that two images are indeed two samples of a particular texture iff their statistics match. The statistics used are histograms of given texture $I$ filtered with a number of filters: $\{hist(F_i * I), \quad i = 1,\dots, k\}$. And whatever image has the same statistics is thought as a sample of texture $I$. The main drawback was the Gibbs sampling was employed (which is very slow). \[4\] suggested exactly the scheme we use now: starting from a random image, let's adjust its statistics iteratively so they match the desired. 

Now, what is changed: the filters. \[4\] used carefully crafted set of filters, and now we use neural network based non-linear filters. We still use the idea of matching statistics, but the statistics improved. 

\[1\] *A.Mahendran, A.Vedaldi [Understanding Deep Image Representations by Inverting Them](https://www.robots.ox.ac.uk/~vgg/publications/2015/Mahendran15/mahendran15.pdf)*

\[2\] *A.Dosovitsky, T.Brox [Inverting Visual Representations with Convolutional Networks](http://arxiv.org/pdf/1506.02753v3.pdf)*

\[3\] *Zhu et. al. Filters, 1997 [Random Fields and Maximum Entropy (FRAME):
Towards a Unified Theory for Texture Modeling](http://www.stat.ucla.edu/~ywu/research/papers/ijcv.pdf)*

\[4\] *Portilla & Simoncelli, 2000  [A Parametric Texture Model Based on Joint Statistics
of Complex Wavelet Coefficients](http://www.cns.nyu.edu/pub/lcv/portilla99-reprint.pdf)*

# Homework

To prevent you from technical problems, you may use a [complete code for the method](https://github.com/Lasagne/Recipes/tree/master/examples/styletransfer). 
Your task will be to play around with it. 

### First part
**Common mandatory part**:
- Generate your favourite texture (please, do not use starry night). All you need to do is to set content weight to 0.  
- Stylize your favourite photo with your favourite style (hope you use something interesting).
- Give an explanation for matching Gram matrices. What does it mean to minimize distance between them in terms of random variables? Assume a true distripution $P$, and model distibution $Q$. What class does $Q$ belong when matching gram matrices? Show, that $KL (P || Q)$ is minimized when Gram matrices are matched. In other words you need to come up with $Q$ such that $KL$ divergence is minimized when models gram matrix is equal to a target Gram matix. If you do not understand the question spend more time, please. If you want a hint after all, here is a [Telegram bot for you](https://telegram.me/rdl_hw7_bot) (send /hint to him).

### Second part
We give you **two options** for the second part.

**First one** (if you are lazy or do not have GPU do just this):
- Implement **Mean** and **Covariance** matching functions instead of $Gram$ matching. That is: 
    - *Mean* is a vector of size `C` which containes means over feature maps
    - *Covariance* matrix is a *Gram* matrix of $Feats-mean$
- What is $Q$ now? 
- Generate texture and stylize with $mean$ loss only; with $mean$ + $Covariance$ loss. Plot the results, side by side (3 textures and 3 stylized). What do you think? Actually, $Gram$ matrix or $Mean$ or $Mean$ + $Covariance$ matrix can be thought as texture descriptors. Does $mean$ encoding have enough parameters to represent texures?  
- ***OR*** come up with your method to remove spatial information instead of above.
- Bonus: you can mix several styles, averaging their representations. It can be fun. Some examples are [here](https://github.com/jcjohnson/neural-style).

**Second one** (hardcore):
- Substitute gram matrices with discriminator as in GAN. That is, you match distributions matching gram matrices and discriminator is designed to match distributions. Probably $Q$, we have defined is weak or too constraintive. Neural network based discriminator should be more flexible in this sense.
    - The procedure will be a little bit unusual: we will optimize NN inside optimization loop w.r.t. image.
    - You need to define a pixel level discriminator (at each layer you have $WH$ objects, each with $C$ features). Basically it should decide whether a pixel came from style image or from current image $X$. 
    - So the process is like that: 
        - At each image optimization iteration update D (actually you do not need to do minibatches updates here, you can simulate fully-connected layers with 1x1 convolutions, softmax with sigmoids). You will need to find a trade-off, for how long and how frequent should updates be. 
        - Then propagate gradient just like in GAN when optimizing $G$ i.e. swap labels (another strategy is in [here](https://www.robots.ox.ac.uk/~vgg/rg/papers/Tzeng_ICCV2015.pdf), eq. 4).
        - Let L-BFGS (or whatever, probably adam will be more stable) update $X$.
    - Discriminator architecture is up to you. It's better to start with logistic regression which should emulate $Mean$ + $Cov$ matching (isn't it?). 
    - I tried this myself without content loss only.

**Do everything in this notebook, I need your code as well as the generated images**

HINTS: 

- In case you do not have GPU, you need to substitute the line:
  
  `from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer`

   with

   `from lasagne.layers import Conv2DLayer as ConvLayer`
   
   
- If you do not have GPU, resize your images to 256x256 and no more. Even at this resolution it may take an hour. You can decrease the number of iterations if it takes too long. 

In [2]:
import sys
import os
import lasagne
import numpy as np
import pickle
import skimage.transform
import scipy

from PIL import Image
import cPickle

import theano
import theano.tensor as T

from lasagne.utils import floatX
from collections import OrderedDict

import matplotlib.pyplot as plt

from IPython import display
%matplotlib inline

from lasagne.layers import InputLayer, DenseLayer, NonlinearityLayer
from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.nonlinearities import softmax

IMAGE_W = 600

# Note: tweaked to use average pooling instead of maxpooling
def build_model():
    net = {}
    net['input'] = InputLayer((1, 3, IMAGE_W, IMAGE_W))
    net['conv1_1'] = ConvLayer(net['input'], 64, 3, pad=1, flip_filters=False)
    net['conv1_2'] = ConvLayer(net['conv1_1'], 64, 3, pad=1, flip_filters=False)
    net['pool1'] = PoolLayer(net['conv1_2'], 2, mode='average_exc_pad')
    net['conv2_1'] = ConvLayer(net['pool1'], 128, 3, pad=1, flip_filters=False)
    net['conv2_2'] = ConvLayer(net['conv2_1'], 128, 3, pad=1, flip_filters=False)
    net['pool2'] = PoolLayer(net['conv2_2'], 2, mode='average_exc_pad')
    net['conv3_1'] = ConvLayer(net['pool2'], 256, 3, pad=1, flip_filters=False)
    net['conv3_2'] = ConvLayer(net['conv3_1'], 256, 3, pad=1, flip_filters=False)
    net['conv3_3'] = ConvLayer(net['conv3_2'], 256, 3, pad=1, flip_filters=False)
    net['conv3_4'] = ConvLayer(net['conv3_3'], 256, 3, pad=1, flip_filters=False)
    net['pool3'] = PoolLayer(net['conv3_4'], 2, mode='average_exc_pad')
    net['conv4_1'] = ConvLayer(net['pool3'], 512, 3, pad=1, flip_filters=False)
    net['conv4_2'] = ConvLayer(net['conv4_1'], 512, 3, pad=1, flip_filters=False)
    net['conv4_3'] = ConvLayer(net['conv4_2'], 512, 3, pad=1, flip_filters=False)
    net['conv4_4'] = ConvLayer(net['conv4_3'], 512, 3, pad=1, flip_filters=False)
    net['pool4'] = PoolLayer(net['conv4_4'], 2, mode='average_exc_pad')
    net['conv5_1'] = ConvLayer(net['pool4'], 512, 3, pad=1, flip_filters=False)
    net['conv5_2'] = ConvLayer(net['conv5_1'], 512, 3, pad=1, flip_filters=False)
    net['conv5_3'] = ConvLayer(net['conv5_2'], 512, 3, pad=1, flip_filters=False)
    net['conv5_4'] = ConvLayer(net['conv5_3'], 512, 3, pad=1, flip_filters=False)
    net['pool5'] = PoolLayer(net['conv5_4'], 2, mode='average_exc_pad')

    return net


# helper functions for all my needs

def prep_image(im):
    if len(im.shape) == 2:
        im = im[:, :, np.newaxis]
        im = np.repeat(im, 3, axis=2)
    h, w, _ = im.shape
    if h < w:
        im = skimage.transform.resize(im, (IMAGE_W, w*IMAGE_W/h), preserve_range=True)
    else:
        im = skimage.transform.resize(im, (h*IMAGE_W/w, IMAGE_W), preserve_range=True)

    # Central crop
    h, w, _ = im.shape
    im = im[h//2-IMAGE_W//2:h//2+IMAGE_W//2, w//2-IMAGE_W//2:w//2+IMAGE_W//2]
    
    rawim = np.copy(im).astype('uint8')
    
    # Shuffle axes to c01
    im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
    
    # Convert RGB to BGR
    im = im[::-1, :, :]

    im = im - MEAN_VALUES
    return rawim, floatX(im[np.newaxis])

def image2pixelarray(filepath):
    im = Image.open(filepath).convert('RGB')
    (width, height) = im.size
    idata = list(im.getdata())
    idata = np.array(idata)
    idata = idata.reshape((height, width, idata.shape[1]))
    return idata

def progress(i, n, skip=100, mode=1):
    if (i%skip == 0 or n < i + skip) and mode == 1:
        if i + 1 < n:
            out = "\r%s%%" % "{:5.2f}".format(100*i/float(n))
        else:
            out = "\r100%"
        sys.stdout.write(out)
        sys.stdout.flush()

def dump_obj(obj, fname):
    try:
        f = file(fname, 'wb')
        cPickle.dump(obj, f, protocol=cPickle.HIGHEST_PROTOCOL)
        f.close()
        return 1
    except Exception as e:
        print e
        return 0

def load_obj(fname):
    try:
        f = file(fname, 'rb')
        loaded_obj = cPickle.load(f)
        f.close()
        return loaded_obj
    except Exception as e:
        print e
        return 0
    
def deprocess(x):
    x = np.copy(x[0])
    x += MEAN_VALUES

    x = x[::-1]
    x = np.swapaxes(np.swapaxes(x, 0, 1), 1, 2)
    
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def do_train(tr_fn, iterations, outfolder, show=True, autosave=100):
    if not os.path.exists(outfolder):
        os.mkdir(outfolder)
    
    for i in range(iterations):
        loss = tr_fn()
        progress(i, iterations, skip=1)
        if not i%autosave:
            cur = generated_image.get_value().astype('float64')
            dump_obj(cur, os.path.join(outfolder,str(u+i)))
            display.clear_output(wait=True)
            
            s = 'iteration: ' + str(u+i) + ' loss: ' + str(loss)
            h.append(s)
            for i in h[-5:]:
                print i
            
            if show:
                plt.figure(figsize=(12,12))
                plt.imshow(deprocess(cur), interpolation='nearest')
                plt.show()
    
MEAN_VALUES = np.array([104, 117, 123]).reshape((3,1,1))

ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device 1 failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1

ERROR:theano.sandbox.cuda:ERROR: Not using GPU. Initialisation of device 1 failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1

Using gpu device 0: Quadro K4200 (CNMeM is enabled with initial size: 30.0% of memory, CuDNN 4007)


In [ ]:
# Download the normalized pretrained weights from:
# https://s3.amazonaws.com/lasagne/recipes/pretrained/imagenet/vgg19_normalized.pkl
# (original source: https://bethgelab.org/deepneuralart/)

!wget https://s3.amazonaws.com/lasagne/recipes/pretrained/imagenet/vgg19_normalized.pkl

In [ ]:
# build VGG net and load weights

net = build_model()
values = pickle.load(open('vgg19_normalized.pkl'))['param values']
lasagne.layers.set_all_param_values(net['pool5'], values)

In [3]:
photo = image2pixelarray('artwork/julie.jpg')
rawim, photo = prep_image(photo)
plt.figure(figsize=(10,10))
plt.imshow(rawim)

IOError: [Errno 2] No such file or directory: 'artwork/julie.jpg'